Script para envio de mensagens simples  
Premisas:  
1. Carregar bibliotecas necessarias
    1. Pandas
    2. Selenium
    3. Webdriver Manager
    4. Time
2. Carregar arquivo de mensagens en dataframe
3. Limpar dataframe  
    1. Eliminar linhas sem mensagens
    2. Eliinar linhas sem telefones
    3. Eliminar colunas desnecessarias
    4. Obter o total de linhas
    5. Resetar indice
4. Abrir o Navegador
5. Abrir o WhatsappWeb
6. Esperar conexão do celular
7. Iniciar Loop de envio de imagens
    1. Enviar mensagem
    2. Iniciar Loop de envio de imagens

# Carregar bibliotecas necessarias

In [1]:
import pandas as pd

# necessary libraries for Chrome operations:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

# modified 29-oct-23
from selenium.webdriver.chrome.service import Service

# pip install webdriver_manager
# This librari updates automatically the Browser Manger (in this case, Chrome)
from webdriver_manager.chrome import ChromeDriverManager

# Necessary to convert messages from ASCII text into URL aceptable addresses (convert special characters, spaces, etc)
import urllib

# Just to get image file name from full path
from pathlib import Path

# Time to allow program wait few seconds during Chrome operations
import time

# To allow randomic waiting times (important to avoid Whatsapp account blocking)
import random

# Datetime to store current date of messages sent
# from datetime import date
import datetime as dt

# importar Tkinter
import tkinter as tk
from tkinter import filedialog as fd

# Defnir Botoes e clicaveis

Nota: ao deixar as definições de objetos clicaveis num arquivo separado, consigo editar caso mude algo sem recompilar

In [2]:
# carregar base de dados de objetos clicaveis
objetos = pd.read_csv("SendWhatsAppMsgObjects.CSV",delimiter=';',encoding='latin-1')

In [3]:
# clicaveis referentes ao envio de mensagens

botao_envio_ccs = objetos.loc[objetos['Object']=='botao_envio_ccs','String'].values[0]
# botão de envio de mensagem

popup_ccs = objetos.loc[objetos['Object']=='popup_ccs','String'].values[0]
# identificador do popup do telefone errado

popup_ok_ccs = objetos.loc[objetos['Object']=='popup_ok_ccs','String'].values[0]
# botão de ok no popup de telefone errado

In [4]:
# clicaveis referentes ao envio de imagens

botao_anexar = objetos.loc[objetos['Object']=='botao_anexar','String'].values[0]
# via CCS Selector
# este é o sinal de "mais" para iniciar o envio de anexos

fotos_e_videos = objetos.loc[objetos['Object']=='fotos_e_videos','String'].values[0]
# via xpath
# NOTA 24-11 este e o caminho certo. Termina em Input. Com isto nao precisa o botao de enviar imagens

botao_enviar_imagens = objetos.loc[objetos['Object']=='botao_enviar_imagens','String'].values[0]
# via CCS Selector
# este é o botão triangulo verde

# Carregar arquivo de mensagens en dataframe

In [5]:
arquivo_de_mensagens = fd.askopenfilename(
    title='Selecione o arquivo Excel com a lista de destinatarios',
    filetypes=[('Arquivo Excel','.xls'),('Arquivo Excel','.xlsx')]
    )

contacts_df = pd.read_excel(arquivo_de_mensagens, sheet_name='CLIENTES')

# Carregar lista de imagens

In [6]:
# selecionar arquivos de imagen

images_types = [
        ('Arquivos de imagen','.jpg'),
        ('Arquivos de imagen','.jpeg'),
        ('Arquivos de imagen','.png'),
        ('Arquivos de imagen','.gif'),
        ]
 
imgs_path = sorted(list(fd.askopenfilenames(title='Selecione as imagens a enviar',filetypes=images_types)))

In [7]:
imgs_path

['C:/SendWhatsAppMessagesV2/Imagens de teste/TESTING IMAGE 1080 X 1350 jpg.jpg',
 'C:/SendWhatsAppMessagesV2/Imagens de teste/TESTING IMAGE 1920 X 1080 png.PNG']

# Limpar dataframe

In [8]:
# 1. Eliminar linhas sem mensagens
contacts_df = contacts_df[~contacts_df['MENSAGEM'].isnull()]

# 2. Eliinar linhas sem telefones
contacts_df = contacts_df[~contacts_df['TELEFONE'].isnull()]

# 3. Reset index
contacts_df.reset_index(inplace=True)

# 4. Eliminar colunas desnecessarias
contacts_df = contacts_df[['CLIENTE','TELEFONE','MENSAGEM']]

# 5. Obter o total de linhas
numero_de_mensagens = contacts_df['MENSAGEM'].count()

print('Serão enviadas {} mensagens'.format(numero_de_mensagens))

Serão enviadas 11 mensagens


In [9]:
# visualizar dataframe
contacts_df

,CLIENTE,TELEFONE,MENSAGEM
0,Antonio,54 9 11 4428-4423,Mensaje de prueba para Antonio\n\nLinea nueva ...
1,Christian,55 (71) 99924-0327,Mensaje de prueba para Christian\n\nLinea nuev...
2,Diego,55 (71) 99956-7221,Mensaje de prueba para Diego\n\nLinea nueva _i...
3,Izabel Wrong Number,55 (555) 99308-2372,Mensaje de prueba para Izabel Wrong Number\n\n...
4,Izabel,55 (71) 99308-2372,Mensaje de prueba para Izabel\n\nLinea nueva _...
5,Lukas,55 (71) 99614-9336,Mensaje de prueba para Lukas\n\nLinea nueva _i...
6,Pablo,55(71) 99101-3035,Mensaje de prueba para Pablo\n\nLinea nueva _i...
7,Pablo Wrong Number,55(111) 99101-3035,Mensaje de prueba para Pablo Wrong Number\n\nL...
8,Rafinha,55 (71) 99622-2775,Mensaje de prueba para Rafinha\n\nLinea nueva ...
9,Rodrigo,54 9 11 4149-4402,Mensaje de prueba para Rodrigo\n\nLinea nueva ...


# Abrir o Navegador

In [10]:
# Criar uma instancia do Google Chrome
msg_browser = webdriver.Chrome()

# Abrir o WhatsappWeb

In [11]:
# Navegar até o WhatsApp Web
msg_browser.get("https://web.whatsapp.com/")
time.sleep(2)

# Esperar conexão do celular

In [12]:
# Esperar pela lista de contatos do WhatsApp por X segundos
# Isto indica que podemos começar a enviar mensagens
while len(msg_browser.find_elements(By.ID,"side")) < 1:
    time.sleep(1)

# Testes

Estes testes funcionaram em 20/11

a maior alteração foi a deteção do botão a traves do aria-label

# Rotina de envio de mensagens

Nota 20-11-2024: utilizando Aria-label no CCS Selector funciona bem.

Nota 24-11-2024: esta serie de clicaveis esta funciondo

Agora temos que verificar algumas melhorias:

1) Encapsular toda a rotina desde o link num try/except, caso de erro, a mensagem nao foi enviada
2) Tal vez utilizar Expected conditions para o envio de imagens
3) Salvar as definições de clicaveis num arquivo, tal vez em CSV, para termos o tipo (Xpath, CCS, etc) e o identificador.
4) Melhorar a saida de registro do Timestamp, tal vez para um dado de tipo data/hora ao inves de string.

In [13]:
for j, mensagem in enumerate(contacts_df['MENSAGEM']):

    # pegar os dados da linha j
    # mensagem = contacts_df.loc[j,'MENSAGEM']
    cliente = contacts_df.loc[j,'CLIENTE']
    telefone = contacts_df.loc[j,'TELEFONE']
        
    # Criar e Abrir Link
    
    # Converter a mensagem de ASCII para texto plano para ser usada como URL
    url_mensagem = urllib.parse.quote(f"{mensagem}")
    
    # Construir o link
    link = f"https://web.whatsapp.com/send?phone={telefone}&text={url_mensagem}"
    
    # aqui teria que ir um try
    try:

        # abrir o link
        msg_browser.get(link)
        
        # Depois do link, esperar ou o botão de envio, ou o pop up de telefone errado
        telefone_errado = False
        envio_disponivel = False

        while not(telefone_errado) and not(envio_disponivel):
            try:
                msg_browser.find_element(By.CSS_SELECTOR,botao_envio_ccs)
                envio_disponivel = True
                # se passou por aqui, quer dizer que o botão de envio esta disponivel
                # print("Encontrei o botão de envio")
            except NoSuchElementException:
                # se não, então o botão de envio não esta disponvivel,
                # e temos que ver se esta o pop up de telefone errado
                # print("Nao encontrei botão de envio, vou tentar o popup")
                try:
                    msg_browser.find_element(By.CSS_SELECTOR,popup_ccs)
                    telefone_errado = True
                    # se estamos aqui, é porque o pop up de telefone errado esta visivel
                    # print("Encontrei o popup")
                except NoSuchElementException:
                    # se estamos aqui, então não encontrou o popup de telefone errado
                    # mas tambem nao encontrou o botao de envio
                    # aqui devemos simplesmente passar para frente
                    # print("Nao encontrei o pop nem o botão de envio")
                    pass
            time.sleep(3)

        # print("Envio disponivel: ",envio_disponivel)
        # print("Telefone errado: ",telefone_errado)
        
        # Depedendo do que foi encontrado, clicar no botao de envio ou no ok de telefone errado
        
        # agora clicar nos botoes correspondetes
        if envio_disponivel:
            # clicar no botáo de envio
            msg_browser.find_element(By.CSS_SELECTOR,botao_envio_ccs).click()
            # Registrar resultado do envio
            contacts_df.loc[j,'RESULTADO'] = 'Mesagem enviada'

            # aqui teriamos que enviar as imagens

            for i, img_file in enumerate(imgs_path):

                # clicar no botão de mais para enviar anexos
                msg_browser.find_element(By.CSS_SELECTOR,botao_anexar).click()
                time.sleep(2)

                # enviar diretamente o caminho da imagem
                msg_browser.find_element(By.XPATH,fotos_e_videos).send_keys(img_file)
                time.sleep(2)

                # clicar no botão de envio de imagem
                msg_browser.find_element(By.CSS_SELECTOR,botao_enviar_imagens).click()
                time.sleep(2)

        else:
            if telefone_errado:
                # clicar no botáo de ok do popup
                msg_browser.find_element(By.CSS_SELECTOR,popup_ok_ccs).click()
                # Registrar resultado do envio
                contacts_df.loc[j,'RESULTADO'] = 'Telefone errado'

    # y aqui el except, caso falle algo
    except:
       contacts_df.loc[j,'RESULTADO'] = 'Erro'

    # Registrar o Timestamp
    contacts_df.loc[j,'TIMESTAMP'] = dt.datetime.now().strftime('%d-%m-%y %H:%M:%S')

    # esperar tempo aleatorio antes da proxima
    time.sleep(random.randint(3,7))   


In [15]:
contacts_df

,CLIENTE,TELEFONE,MENSAGEM,RESULTADO,TIMESTAMP
0,Antonio,54 9 11 4428-4423,Mensaje de prueba para Antonio\n\nLinea nueva ...,Mesagem enviada,27-11-24 19:11:08
1,Christian,55 (71) 99924-0327,Mensaje de prueba para Christian\n\nLinea nuev...,Mesagem enviada,27-11-24 19:11:41
2,Diego,55 (71) 99956-7221,Mensaje de prueba para Diego\n\nLinea nueva _i...,Mesagem enviada,27-11-24 19:12:12
3,Izabel Wrong Number,55 (555) 99308-2372,Mensaje de prueba para Izabel Wrong Number\n\n...,Telefone errado,27-11-24 19:12:25
4,Izabel,55 (71) 99308-2372,Mensaje de prueba para Izabel\n\nLinea nueva _...,Mesagem enviada,27-11-24 19:12:52
5,Lukas,55 (71) 99614-9336,Mensaje de prueba para Lukas\n\nLinea nueva _i...,Mesagem enviada,27-11-24 19:13:21
6,Pablo,55(71) 99101-3035,Mensaje de prueba para Pablo\n\nLinea nueva _i...,Mesagem enviada,27-11-24 19:13:51
7,Pablo Wrong Number,55(111) 99101-3035,Mensaje de prueba para Pablo Wrong Number\n\nL...,Telefone errado,27-11-24 19:14:06
8,Rafinha,55 (71) 99622-2775,Mensaje de prueba para Rafinha\n\nLinea nueva ...,Mesagem enviada,27-11-24 19:14:33
9,Rodrigo,54 9 11 4149-4402,Mensaje de prueba para Rodrigo\n\nLinea nueva ...,Mesagem enviada,27-11-24 19:15:03


In [19]:
contacts_df.drop('MENSAGEM',axis=1).to_excel('Resultados.xlsx')